<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('./toc.js')

## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import rc
import matplotlib
import numpy as np
from tabulate import tabulate
import re
import pandas as pd
import pathlib
import pprint # For pretty printing

outputRoot = pathlib.Path('output')

## Conventions

In [ ]:
CSV_SEP = ";"
DPI_DEFAULT = 150
DEFAULT_FONT_SIZE = 11
DEFAULT_HEIGHT = 3
DEFAULT_TIGHT_LAYOUT_PADDING = 0

# Color codes
# #1: 68,84,106
# #2: 237,125,49
COLORS = ['#44546a', '#ed7d31', '#2ca25f']

linewidth = 2.0


params = {'text.usetex' : False,
      'font.size' : DEFAULT_FONT_SIZE,
      'font.family' : 'CMU Serif',
      }
plt.rcParams.update(params)
matplotlib.rcParams['mathtext.fontset'] = 'cm'

matplotlib.rcParams["figure.figsize"][1] = DEFAULT_HEIGHT

## Helper functions for boxplots

In [ ]:
BXP_MEDIANS = 'medians'
BXP_FLIERS = 'fliers'
BXP_CAPS = 'caps'
BXP_WHISKERS = 'whiskers'
BXP_BOXES = 'boxes'

def setBoxplotColor(bp, color, subplotIndices=None):
  if subplotIndices is None:
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['medians'], color=color)
  else:
    for index in subplotIndices:
      plt.setp(bp['boxes'][index], color=color)
      plt.setp(bp['medians'][index], color=color)
  for box in bp['boxes']:
    box.set_facecolor('white')

""" Returns the value of the given property from the given boxplot
@param property: one of [BXP_MEDIANS, BXP_CAPS, BXP_WHISKERS, BXP_FLIERS]
@param boxplot: a boxplot as returned by axes.boxplot in matplotlib
"""
def getBoxplotProperty(boxplot, property):
  return np.array([item.get_ydata() for item in boxplot[property]])

def formatBoxplotPropertyValues(boxplot, property):
  formatted_entries = CSV_SEP.join([formatBoxplotPropertyValue(property, entry) for entry in getBoxplotProperty(boxplot, property)])
  cleaned_result = list(formatted_entries)
  if property in [BXP_CAPS, BXP_WHISKERS]:
    idx = -1
    while True:
      idx = idx + 1
      idx = formatted_entries.find(CSV_SEP, idx)
      if idx < 0:
        break

      cleaned_result[idx] = ","

      idx = idx + 1
      idx = formatted_entries.find(CSV_SEP, idx)
      if idx < 0:
        break
  return "".join(cleaned_result)

def formatBoxplotPropertyValue(property, entry):
  if len(entry) != 2:
    return "n/a"
  elif property in [BXP_MEDIANS, BXP_CAPS]:
    return "%.3g" % (entry[0])
  else:
    return "%.3g - %.3g" % (entry[0], entry[1])

"""
Writes key values of the given boxplot to the given output_file.
"""
def writeBoxplotStatistics(boxplot, output_file, metric, append=False):
  newline = "\n"
  fileMode = 'a' if append else 'w'
  with open(output_file, mode=fileMode, newline=newline) as statistics_file:
    
    line = "%s;%s;%s" % ("property", 
                         "metric", 
                         CSV_SEP.join(["col_%02d" % i for i,entry in enumerate(getBoxplotProperty(boxplot, BXP_MEDIANS))]))
    statistics_file.write(line)
    statistics_file.write(newline)
    
    for property in [BXP_MEDIANS, BXP_CAPS, BXP_WHISKERS, BXP_FLIERS]:
      line = "%s;%s;%s" % (property, metric, formatBoxplotPropertyValues(boxplot, property))
      statistics_file.write(line)
      statistics_file.write(newline)
    
    statistics_file.write(newline)

def shallSkipBoxplotElement(element, elementEntryNumber, subplotIndices):
  if subplotIndices is None:
    return False
  elif element in [BXP_MEDIANS]:
    # Single element per boxplot
    return elementEntryNumber not in subplotIndices
  else:
    # Two elements per boxplot
    return elementEntryNumber // 2 not in subplotIndices
        
def annotateBoxplot(boxplot, axes, 
               labeledBoxplotElements = [BXP_MEDIANS], 
               enablePadding=True, relativePadding=7, subplotIndices=None,
               valueFormatter='{:,.0f}'):
  """
  Labels the median, quartiles and whiskers of a boxplot

  How to use:
  fig, axes = plt.subplots()
  boxplot = axes.boxplot(boxplot_data, showfliers=True)
  annotateBoxplot(boxplot, axes)

  Thanks to https://stackoverflow.com/a/47475890/6844815
  
  @param labeledBoxplotElements a sublist of [BXP_MEDIANS, BXP_CAPS, BXP_WHISKERS, BXP_FLIERS]
  """
  
  # Extent of plot (as bounding box)
  extent=axes.get_window_extent()
  # Determine how many data units correspond to one pixel
  unitsPerPixel= 1.0 * (axes.get_ylim()[1] - axes.get_ylim()[0]) / (extent.y1 - extent.y0)
  
  for element in labeledBoxplotElements:
    for entryNumber, line in enumerate(boxplot[element]):
      
      if shallSkipBoxplotElement(element, entryNumber, subplotIndices):
        continue
              
      # Get the position of the element. y is the label you want
      (x_l, y),(x_r, _) = line.get_xydata()
      
      # Make sure datapoints exist
      if not np.isnan(y):
        x_line_center = x_l + (x_r - x_l)/2
        y_line_center = y  # Since it's a line and it's horisontal
        
        verticalAlignmentSwitcher = {
          BXP_MEDIANS: 'bottom',
          BXP_CAPS: 'top' if entryNumber % 2 == 0 else 'bottom',
          BXP_WHISKERS: 'top' if entryNumber % 2 == 0 else 'bottom',
          BXP_FLIERS: 'top' if entryNumber % 2 == 0 else 'bottom'
        }
        
        horizontalAlignmentSwitcher = {
          BXP_MEDIANS : 'center',
          BXP_CAPS: 'center',
          BXP_WHISKERS: 'left',
          BXP_FLIERS: 'center'
        }
        
        verticalAlignment=verticalAlignmentSwitcher[element]
        horizontalAlignment=horizontalAlignmentSwitcher[element]          
        value = valueFormatter.format(y)
        
        if enablePadding:
          if horizontalAlignment == 'left':
            value = ' ' + value

          # How much the text shall be lowered / raised (in pixel)
          offsetInPixel = relativePadding * line.get_linewidth()

          if verticalAlignment == 'top':
            y_line_center = y_line_center - offsetInPixel * unitsPerPixel
          elif verticalAlignment == 'bottom':
            y_line_center = y_line_center + offsetInPixel * unitsPerPixel
        
        axes.text(x_line_center, y_line_center, # Position
                  value, # Value (3f = 3 decimal float)
                  verticalalignment=verticalAlignment, # 'center', 'top', 'bottom'
                  horizontalalignment=horizontalAlignment, # 'center', 'left', 'right'
                  fontsize=DEFAULT_FONT_SIZE,
                  backgroundcolor="#00000000",
                  #bbox=dict(boxstyle="square,pad=5",fill=None,linewidth=0),
                 )

## Helper functions for ```tabulate```

In [ ]:
def compressWhitespace(inputFormattedTable):
  return re.sub("[ ]+", " ", inputFormattedTable)
  
def convertToBooktabs(inputFormattedTable):
  outputFormattedTable = inputFormattedTable
  outputFormattedTable = outputFormattedTable.replace("\\hline", "\\toprule", 1)
  outputFormattedTable = outputFormattedTable.replace("\\hline", "\\midrule", 1)
  outputFormattedTable = outputFormattedTable.replace("\\hline", "\\bottomrule", 1)
  return outputFormattedTable
  
def encloseWithTableEnvironment(formattedTable, caption, label, shortCaption=None):
  optionalShortCaption = "" if shortCaption is None else "[" + shortCaption + "]"
  return """\\begin{table}
\\begin{center}
\\caption%s{%s}
\\label{%s}
%s
\\end{center}
\\end{table}
""" % (optionalShortCaption, caption, label, formattedTable)

def textbf(string):
  return "\\textbf{%s}" % string

def adjustAlignment(formattedTable, alignmentSpecification):
  return re.sub(r"(\\begin\{tabular\}\{).*(\})", r"\1" + alignmentSpecification + r"\2", formattedTable)

def createMidrule():
  return r"\midrule" + "\n"

## Helper functions for file system

In [ ]:
def getNormalizedPathName(path):
  return str(path.absolute()).replace("\\", "/")

def showCsvFile(path):
  data = pd.read_csv(path, sep=CSV_SEP)
  print("Content of " + str(path))
  print(data)

## $\LaTeX$ commands

In [ ]:
commandsFromJupyterPath = outputRoot / "commandsFromJupyter.tex"
commandMap = dict()

def registerLatexCommand(commandName, value):
  
  commandMap[commandName] = value
  
  sortedKeys = sorted(commandMap.keys())
    
  with open(str(commandsFromJupyterPath), "w") as outputFile:
    outputFile.write("% This file contains latex command generated using Jupyter Notebook\n")
    for key in sortedKeys:
      outputFile.write("\\def%s{%s\\mySpace}\n" % (key, commandMap[key]))
  
def siPercentage(value, formatter="%.1f"):
  formatString = "\\SI{%s}{\\percent}" % formatter
  return formatString % value

def siMeter(value, formatter="%.0f"):
  formatString = "\\SI{%s}{\\meter}" % formatter
  return formatString % value

def num(value):
  return "\\num{%d}" % value


# cMoflon

## Prepare output folder for cMoflon results

In [ ]:
cmoflonOutputFolder = outputRoot / 'cmoflon'
cmoflonOutputFolder.mkdir(exist_ok=True,parents=True)

## Size of cMoflon

In [ ]:
cMoflonStringTemplateSizeData = pd.read_csv('data/cmoflon/linesOfStg.csv', sep=CSV_SEP)

print(cMoflonStringTemplateSizeData.columns)

cMoflonStringTemplateSizeDataEMoflon = cMoflonStringTemplateSizeData[cMoflonStringTemplateSizeData['tool'] == 'emoflon']
cMoflonStringTemplateSizeDataCMoflon = cMoflonStringTemplateSizeData[cMoflonStringTemplateSizeData['tool'] == 'cmoflon']

furtherSizeInformation = {
"linesOfStCodeEMoflon" : sum(cMoflonStringTemplateSizeDataEMoflon['count']),
"numberOfStgFilesEMoflon" : len(cMoflonStringTemplateSizeDataEMoflon),
"linesOfStCodeCMoflon" : sum(cMoflonStringTemplateSizeDataCMoflon['count']),
"numberOfStgFilesCMoflon" : len(cMoflonStringTemplateSizeDataCMoflon),
"locGeneratedCodeEMoflon" : 510801
}


cMoflonLinesOfCodeData = pd.DataFrame(
  [[r"\toolEMoflon", 64, 568124, 35622, furtherSizeInformation["linesOfStCodeEMoflon"]],
   [r"\toolDemocles", 11, 8676, 0, 0],
   [r"\toolCMoflon", 5, 3797, 0,  furtherSizeInformation["linesOfStCodeCMoflon"]]], 
  columns=["Tool", "Projects", r"LOC \toolJava", r"LOC \toolCSharp", r"LOC ST"])

print("Further size information:")
pprint.pprint(furtherSizeInformation)

In [ ]:
cMoflonLinesOfCodeDataForFormatting = cMoflonLinesOfCodeData
for numericColumn in cMoflonLinesOfCodeData.columns[1:]:
  cMoflonLinesOfCodeDataForFormatting[numericColumn] = \
    [num(value) for value in cMoflonLinesOfCodeDataForFormatting[numericColumn]]

formattedTable=tabulate(cMoflonLinesOfCodeDataForFormatting, 
               headers=([textbf(name) for name in cMoflonLinesOfCodeDataForFormatting.columns]),
               showindex = None,
               tablefmt="latex_raw")

formattedTable = convertToBooktabs(formattedTable)
formattedTable = compressWhitespace(formattedTable)
formattedTable = adjustAlignment(formattedTable, alignmentSpecification="lrrrr")
formattedTable = encloseWithTableEnvironment(formattedTable, 
                                             r"Size comparison \toolEMoflon \vs \toolCMoflon (ST: \toolStringTemplate, LOC: lines of code)", 
                                             "tabLinesOfCodeEMoflonVsCMoflon",
                                            shortCaption=r"Size comparison \toolEMoflon \vs \toolCMoflon")

print(formattedTable)

In [ ]:
formattedTablePath = cmoflonOutputFolder / "linesOfCode.tex"

with open(str(formattedTablePath), "w") as outputFile:
  outputFile.write(formattedTable)
    
del formattedTablePath
del formattedTable

## Binary image size

In [ ]:
rawSensorSizeData = pd.read_csv('data/cmoflon/size.csv', sep=CSV_SEP)
imageToAlgorithmName= {"Maxpower.sky" : "MP", 
                      "KtcMan.sky" : "$\\tcaKtc_{\\idxFont{M}}$",
                      "KtcGen.sky" : "$\\tcaKtc_{\\idxFont{G}}$",
                      "LStarKtcMan.sky" : "$\\tcaLStarKtc_{\\idxFont{M}}$",
                      "LStarKtcGen.sky" : "$\\tcaLStarKtc_{\\idxFont{G}}$",
                      "LmstMan.sky" : "$\\tcaLmst_{\\idxFont{M}}$",
                      "LmstGen.sky" : "$\\tcaLmst_{\\idxFont{G}}$",
                      }

print(rawSensorSizeData)

In [ ]:
def fraction(actualValue, baseValue):
    return "\\SI[retain-explicit-plus]{%.1f}{\\percent}" % (100.0 * actualValue / baseValue - 100)

processedSensorSizeData = list()
placeholderNotAvailable = r"\multicolumn{1}{c}{--}"

# Assumed order
# Line 0: Maxpower algorithm
# Line 1: Algorithm 1, manual
# Line 2: Algorithm 1, generated
# ...
for i,row in rawSensorSizeData.iterrows():
  latexAlgo = imageToAlgorithmName[row['filename']]
  
  if i in [1,3,5]:
    latexAlgo = createMidrule() + latexAlgo
  
  textSize = row['text']
  dataSize = row['data']
  bssSize = row['bss']
  totalSize = row['dec']
  if i == 0:
    textMaxpowerFraction = placeholderNotAvailable
    dataMaxpowerFraction = placeholderNotAvailable
    bssMaxpowerFraction = placeholderNotAvailable
    totalMaxpowerFraction= placeholderNotAvailable

    textManGenFraction = placeholderNotAvailable
    dataManGenFraction = placeholderNotAvailable
    bssManGenFraction = placeholderNotAvailable
    totalManGenFraction = placeholderNotAvailable
  else:
    firstRow = rawSensorSizeData.head(1)
    textMaxpowerFraction = fraction(textSize, firstRow['text'])
    dataMaxpowerFraction = fraction(dataSize, firstRow['data'])
    bssMaxpowerFraction = fraction(bssSize, firstRow['bss'])
    totalMaxpowerFraction = fraction(totalSize, firstRow['dec'])
    del firstRow

  if i % 2 == 1:
    textManGenFraction = placeholderNotAvailable
    dataManGenFraction = placeholderNotAvailable
    bssManGenFraction = placeholderNotAvailable
    totalManGenFraction = placeholderNotAvailable
  elif i >= 2:
    previousRowIndex = i-1
    textManGenFraction = fraction(textSize, rawSensorSizeData['text'][previousRowIndex])
    dataManGenFraction = fraction(dataSize, rawSensorSizeData['data'][previousRowIndex])
    bssManGenFraction = fraction(bssSize, rawSensorSizeData['bss'][previousRowIndex])
    totalManGenFraction = fraction(totalSize, rawSensorSizeData['dec'][previousRowIndex])
    del previousRowIndex

  newRow = [latexAlgo,
            num(textSize), textManGenFraction, #textMaxpowerFraction,
            num(dataSize), dataManGenFraction, #dataMaxpowerFraction,
            num(bssSize), bssManGenFraction, #bssMaxpowerFraction,
            num(totalSize), totalManGenFraction, #totalMaxpowerFraction,
           ]
  processedSensorSizeData.append(newRow)
    
formattedTable=tabulate(processedSensorSizeData, 
               headers=("\\textbf{Algorithm}",
                        "\\textbf{Text[B]}",  "\\deltaManGen" , #"\\deltaMaxpower",
                        "\\textbf{Data[B]}",  "\\deltaManGen" , #"\\deltaMaxpower",
                        "\\textbf{BSS[B]}",   "\\deltaManGen" , #"\\deltaMaxpower",
                        "\\textbf{Total[B]}", "\\deltaManGen" , #"\\deltaMaxpower"
                       ),
               tablefmt="latex_raw")

formattedTable = convertToBooktabs(formattedTable)
formattedTable = compressWhitespace(formattedTable)
formattedTable = adjustAlignment(formattedTable, 'l' + 8 * 'r')
formattedTable = encloseWithTableEnvironment(formattedTable, 
                                             "Binary image size comparison (\\deltaManGen: Manual-to-generated size increase)", 
                                             "tabCMoflonBinaryImageSizes",
                                            shortCaption="Binary image size comparison")

print(formattedTable)

formattedSensorSizeTablePath = cmoflonOutputFolder / "sensorSizes.tex"

with open(str(formattedSensorSizeTablePath), "w") as outputFile:
    outputFile.write(formattedTable)

## Runtime of generated algorithms

In [ ]:
rawRuntimeData = pd.read_csv('data/cmoflon/runtime.csv', sep=CSV_SEP)

rawRuntimeDataLStarKtcGen = pd.read_csv('data/cmoflon/runtimeLStarKtcGenImproved.csv', sep=CSV_SEP)

In [ ]:
def extractBoxplotData(dataframe, algorithm):
  subset = dataframe[dataframe['algo']==algorithm]
  return subset['runtimeMillis']


dataLStarKtcGenImproved = extractBoxplotData(rawRuntimeDataLStarKtcGen, "LStarKtcAlgorithm")

manualKeys = ["aktc", "lktc", "lmst"]
generatedKeys = ["KtcAlgorithm", "LStarKtcAlgorithm", "LmstAlgorithm"]

dataManual = [extractBoxplotData(rawRuntimeData, key) for key in manualKeys]
dataGenerated = [extractBoxplotData(rawRuntimeData, key) for key in generatedKeys]
dataLStarKtcGenOriginal = dataGenerated[1]
dataLStarKtcMan = dataManual[1]
dataGenerated[1] = dataLStarKtcGenImproved

ticks = ['kTC', 'l*kTC', 'LMST']
ticks = ["%s\n(N=%d/%d) " % (tick, len(dataManual[i]), len(dataGenerated[i])) for i, tick in enumerate(ticks)]

figsize = matplotlib.rcParams["figure.figsize"]
figsize[1] = 4
figure = plt.figure(dpi=DPI_DEFAULT,figsize=figsize)

bpl = plt.boxplot(dataManual, positions=np.array(range(len(dataManual)))*2.0-0.4, sym='', patch_artist=True)
bpr = plt.boxplot(dataGenerated, positions=np.array(range(len(dataGenerated)))*2.0+0.4, sym='', patch_artist=True)
setBoxplotColor(bpl, COLORS[0]) 
setBoxplotColor(bpr, COLORS[1], subplotIndices=[0,2])
setBoxplotColor(bpr, COLORS[2], subplotIndices=[1])

# Draw temporary red and blue lines and use them to create a legend
# Temporary variable is necessary to avoid duplicate plots in Jupyter
_ = plt.plot([], c=COLORS[0], label='Manual')
_ = plt.plot([], c=COLORS[1], label='Generated')
_ = plt.plot([], c=COLORS[2], label='Adjusted')
plt.legend(loc='upper left')

annotateBoxplot(bpl, figure.axes[0], relativePadding=-2, subplotIndices=[0])
annotateBoxplot(bpl, figure.axes[0], relativePadding=2, subplotIndices=[1,2])
annotateBoxplot(bpr, figure.axes[0], relativePadding=2)

axes = plt.axes()
axes.yaxis.grid()
plt.xticks(range(0, len(ticks) * 2, 2), ticks)
plt.xlim(-1, len(ticks)*2-1)
plt.ylim(-1, axes.get_ylim()[1])
plt.xlabel("Algorithm")
plt.ylabel("Execution time [ms]")

plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
plt.show()

basename = "plotCMoflonRuntimeOverview"
plotsFile = cmoflonOutputFolder / ('%s.pdf' % basename)
statisticsFile = cmoflonOutputFolder / ('%s.txt' % basename)

print("Write plots to %s" % getNormalizedPathName(plotsFile))
figure.savefig(filename=str(plotsFile))
plt.close()

print("Write statistics to %s" % getNormalizedPathName(statisticsFile))
writeBoxplotStatistics(bpl, statisticsFile, "Runtime [ms]")
writeBoxplotStatistics(bpr, statisticsFile, "Runtime [ms]", append=True)

showCsvFile(statisticsFile)

## Runtime of improved LStarKtcGen

In [ ]:
dataLeft = [dataLStarKtcMan, dataLStarKtcGenOriginal, dataLStarKtcGenImproved]

ticks = ['l*kTC manual', 'l*kTC original', 'l*kTC adjusted']
ticks = ["%s\n(N=%d) " %(tick, len(dataLeft[i])) for i, tick in enumerate(ticks)]

figure = plt.figure(dpi=DPI_DEFAULT)

bpl = plt.boxplot(dataLeft, positions=np.array(range(len(dataLeft))), sym='', widths=0.4, patch_artist=True)
setBoxplotColor(bpl, COLORS[0], subplotIndices=[0]) 
setBoxplotColor(bpl, COLORS[1], subplotIndices=[1]) 
setBoxplotColor(bpl, COLORS[2], subplotIndices=[2]) 

_ = plt.plot([], c=COLORS[0], label='Manual')
_ = plt.plot([], c=COLORS[1], label='Generated')
_ = plt.plot([], c=COLORS[2], label='Adjusted')
plt.legend(loc='upper left')

annotateBoxplot(bpl, figure.axes[0], relativePadding=0, subplotIndices=[0])
annotateBoxplot(bpl, figure.axes[0], relativePadding=0, subplotIndices=[1])
annotateBoxplot(bpl, figure.axes[0], relativePadding=-.7, subplotIndices=[2])

axes = plt.axes()
axes.yaxis.grid()
plt.xticks(range(0, len(ticks)), ticks)
plt.xlim(-.5, len(ticks) - .5)
plt.ylim(3,1.5*max(dataLeft[1]))
plt.yscale("log")
plt.xlabel("Algorithm")
plt.ylabel("Execution time [ms]")

plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
plt.show()

basename = "plotCMoflonLStarKtcGenComparison"
plotFile = cmoflonOutputFolder / ('%s.pdf' % basename)
statisticsFile = cmoflonOutputFolder / ('%s.txt' % basename)

print("Write plots to %s" % getNormalizedPathName(plotFile))
figure.savefig(filename=str(plotFile))
plt.close()

print("Write statistics to %s" % getNormalizedPathName(statisticsFile))
writeBoxplotStatistics(bpl, statisticsFile, "Execution time [ms]")
showCsvFile(statisticsFile)

# Clean up
del bpl
del figure
del dataLeft
del ticks

## LMST tree size (overview of data)

In [ ]:
treeSizeRawData = pd.read_csv('data/cmoflon/treeSize.csv', sep=CSV_SEP)

In [ ]:
treeSizeData = [treeSizeRawData['treeEntrySizeAll']]

ticks = ['Tree size[B]']


figure = plt.figure(dpi=DPI_DEFAULT)

bpl = plt.boxplot(treeSizeData, positions=np.array(range(len(treeSizeData))), sym='', widths=0.4, patch_artist=True)
setBoxplotColor(bpl, COLORS[0]) 

# draw temporary red and blue lines and use them to create a legend
# Temporary variable is necessary to avoid duplicate plots in Jupyter
# _ = plt.plot([], c=COLORS[0], label='Generated')
# _ = plt.plot([], c=COLORS[1], label='Improved')
# plt.legend()
annotateBoxplot(bpl, figure.axes[0], labeledBoxplotElements=[BXP_MEDIANS, BXP_CAPS, BXP_WHISKERS])

plt.xticks(range(0, len(ticks)), ticks)
plt.xlim(-1, len(ticks))
plt.ylim(1,1.2 * max(treeSizeData[0]))
plt.ylabel("Runtime [ms]")

plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
plt.show()

basename = "plotCMoflonTreeSize"
plotFile = cmoflonOutputFolder / ('%s.pdf' % basename)
statisticsFile = cmoflonOutputFolder / ('%s.txt' % basename)

print("Write plots to %s" % getNormalizedPathName(plotFile))
figure.savefig(filename=str(plotFile))
plt.close()

print("Write statistics to %s" % getNormalizedPathName(statisticsFile))
writeBoxplotStatistics(bpl, statisticsFile, "Runtime [ms]")
showCsvFile(statisticsFile)

del bpl 
del basename
del plotFile
del statisticsFile

## Input topology size (overview of data)

In [ ]:
inputTopologySizeRawData = pd.read_csv('data/cmoflon/inputTopologySize.csv', sep=CSV_SEP)

In [ ]:
topologySizeData = [inputTopologySizeRawData['topologySize']]

ticks = [""]


figure = plt.figure(dpi=DPI_DEFAULT)

bpl = plt.boxplot(topologySizeData, positions=np.array(range(len(topologySizeData))), sym='', widths=0.4, patch_artist=True)
setBoxplotColor(bpl, COLORS[0]) 

# draw temporary red and blue lines and use them to create a legend
# Temporary variable is necessary to avoid duplicate plots in Jupyter
# _ = plt.plot([], c=COLORS[0], label='Generated')
# _ = plt.plot([], c=COLORS[1], label='Improved')
# plt.legend()
annotateBoxplot(bpl, figure.axes[0], labeledBoxplotElements=[BXP_MEDIANS, BXP_CAPS, BXP_WHISKERS])

plt.xticks(range(0, len(ticks)), ticks)
plt.xlim(-1, len(ticks))
plt.ylim(min(topologySizeData[0]),1.1 * max(topologySizeData[0]))
plt.ylabel('Topology size[node + edge count]')

plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
plt.show()

# Cobolt

## Prepare output folder for Cobolt results

In [ ]:
coboltOutputFolder = pathlib.Path('output') / 'cobolt'
coboltOutputFolder.mkdir(exist_ok=True,parents=True)

## Read simulation results

In [ ]:
coboltData = pd.read_csv('data/cobolt/dataCollected.csv', sep=CSV_SEP)
uniqueIds = coboltData.uniqueId.unique()

In [ ]:
coboltData.columns

## Aggregate data per experiment

In [ ]:
movementSpeedPerExperiment = pd.read_csv("data/cobolt/movementSpeedPerExperiment.csv", sep=CSV_SEP)
# movementSpeedPerExperiment

In [ ]:
CB_ID = 'uniqueId'
CB_WORLD_SIZE = 'worldSize'
CB_MOTE_COUNT = 'moteCount'
CB_MOVEMENT_SPEED = 'movementSpeed'
CB_MOTE_COUNT_EMPTY = 'moteCountEmpty'
CB_LINK_COUNT_INITIAL = 'linkCountInitial'
CB_DENSITY_INITIAL = 'densityInitial'
CB_DENSITY_INITIAL_BUCKET = 'densityInitialBucket'
CB_TC_OPERATION_MODE = 'tcOperationMode'
CB_DURATION_SIM = 'totalSimulationDurationSeconds',
CB_DURATION_TCA = 'totalTcaDurationSeconds'
CB_DURATION_CEH = 'totalCehDurationSeconds'
CB_DURATION_TCC = 'totalTccDurationSeconds'
CB_LSM_TCA = 'totalTcaLsm'
CB_LSM_CEH = 'totalCehLsm'
CB_LSM_ALL = 'totalAllLsm'

aggregatedCoboltData = pd.DataFrame(columns=[CB_ID, 
                                    CB_WORLD_SIZE,
                                    CB_MOTE_COUNT,
                                    CB_MOTE_COUNT_EMPTY,
                                    CB_LINK_COUNT_INITIAL,
                                    CB_DENSITY_INITIAL,
                                    CB_DENSITY_INITIAL_BUCKET,
                                    CB_TC_OPERATION_MODE,
                                    CB_MOVEMENT_SPEED,
                                    CB_DURATION_SIM, 
                                    CB_DURATION_TCA, 
                                    CB_DURATION_CEH,
                                    CB_DURATION_TCC,
                                    CB_LSM_TCA, 
                                    CB_LSM_CEH,
                                    CB_LSM_ALL,
                                   ])

for uniqueId in uniqueIds:
  selector = coboltData[CB_ID]==uniqueId
  dataForUniqueId = coboltData[selector]
  
  tcOperationMode = dataForUniqueId['cAlgoMode'].unique()
  if len(tcOperationMode) > 1:
    raise Exception("Operation mode not unique for %d" % uniqueId)
    
  lsmCountTca = sum(dataForUniqueId['tcLSMCountEffective'])
  lsmCountCeh = sum(dataForUniqueId['ceLSMCountEffective'])
  # Remove 1 from counts for the base station
  moteCountInitial = dataForUniqueId['cNodeCount'].values[0] - 1
  moteCountEmpty = max(dataForUniqueId['nodeCountEmpty'])
  linkCountInitial = dataForUniqueId['edgeCountTotal'].values[0]
  
  densityInitial = linkCountInitial / moteCountInitial
  bucketSize = 10
  densityInitialBucket = int(densityInitial / bucketSize) * bucketSize
  
  durationSimulationSeconds = 60 * max(dataForUniqueId['totalTimeInMinutes'])
  durationTcaSeconds = sum(dataForUniqueId['tcTimeInMillis'] / 1000)
  durationCehSeconds = sum(dataForUniqueId['ceTimeInMillis'] / 1000)
  movementSpeed = min((movementSpeedPerExperiment[movementSpeedPerExperiment[CB_ID]==uniqueId])['speed'])
  aggregatedCoboltData.loc[len(aggregatedCoboltData)] = {
    CB_ID : uniqueId,
    CB_WORLD_SIZE : dataForUniqueId['cWorldSize'].unique()[0],
    CB_MOTE_COUNT : moteCountInitial,
    CB_MOTE_COUNT_EMPTY : moteCountEmpty,
    CB_LINK_COUNT_INITIAL : linkCountInitial,
    CB_DENSITY_INITIAL : densityInitial,
    CB_DENSITY_INITIAL_BUCKET : densityInitialBucket,
    CB_TC_OPERATION_MODE : tcOperationMode[0],
    CB_MOVEMENT_SPEED : movementSpeed,
    CB_DURATION_SIM : durationSimulationSeconds,
    CB_DURATION_TCA : durationTcaSeconds, 
    CB_DURATION_CEH : durationCehSeconds,
    CB_DURATION_TCC : durationTcaSeconds + durationCehSeconds,
    CB_LSM_TCA  : lsmCountTca, 
    CB_LSM_CEH  : lsmCountCeh, 
    CB_LSM_ALL : lsmCountTca + lsmCountCeh,
    }
  
worldSizeSmall = min(aggregatedCoboltData[CB_WORLD_SIZE])
worldSizeMedium = aggregatedCoboltData[CB_WORLD_SIZE].median()
worldSizeLarge = max(aggregatedCoboltData[CB_WORLD_SIZE])

moteCountFew = min(aggregatedCoboltData[CB_MOTE_COUNT])
moteCountMany = max(aggregatedCoboltData[CB_MOTE_COUNT])

registerLatexCommand(r"\coboltWorldSizeSmall", siMeter(worldSizeSmall))
registerLatexCommand(r"\coboltWorldSizeMedium", siMeter(worldSizeMedium))
registerLatexCommand(r"\coboltWorldSizeLarge", siMeter(worldSizeLarge))
registerLatexCommand(r"\coboltMoteCountFew", num(moteCountFew))
registerLatexCommand(r"\coboltMoteCountMany", num(moteCountMany))

aggregatedCoboltData.to_csv('data/cobolt/dataAggregatedPerExperiment.csv', index=False, sep=CSV_SEP)

In [ ]:
aggregatedCoboltDataBatch=aggregatedCoboltData[aggregatedCoboltData[CB_TC_OPERATION_MODE]=='B']
aggregatedCoboltDataDynamic=aggregatedCoboltData[aggregatedCoboltData[CB_TC_OPERATION_MODE]=='I']

## Simulation-Cobolt runtime comparison

In [ ]:
coboltRuntimeDataForBoxplot = [aggregatedCoboltData[CB_DURATION_SIM], 
                               aggregatedCoboltData[CB_DURATION_TCA],
                               aggregatedCoboltData[CB_DURATION_CEH],
                               ]

ticks = ["Simulation\n\n(N=%d)" % len(aggregatedCoboltData[CB_DURATION_SIM]),
         "Cobolt\nTC algorithm\n(N=%d)" % len(aggregatedCoboltData[CB_DURATION_TCA]), 
         "Cobolt\nContext events\n(N=%d)" % len(aggregatedCoboltData[CB_DURATION_CEH]), 
        ]

figsize = matplotlib.rcParams["figure.figsize"]
figsize[1] = 3
w, h = plt.figaspect(.6)
figure = plt.figure(dpi=DPI_DEFAULT, figsize=(w, h))
del figsize

bpl = plt.boxplot(coboltRuntimeDataForBoxplot, 
                  positions=np.array(range(len(coboltRuntimeDataForBoxplot))), 
                  sym='', widths=0.3, patch_artist=True)
setBoxplotColor(bpl, COLORS[0]) 

annotateBoxplot(bpl, figure.axes[0], labeledBoxplotElements=[BXP_MEDIANS], 
                subplotIndices=[0],valueFormatter='{:.1f}')
annotateBoxplot(bpl, figure.axes[0], labeledBoxplotElements=[BXP_MEDIANS], 
                subplotIndices=[1,2], relativePadding=5,valueFormatter='{:.1f}')


axes = plt.axes()
axes.yaxis.grid()
plt.xticks(range(0, len(ticks)), ticks)
plt.xlim(-.4, len(ticks) - .6)
plt.ylabel('Execution time [s]')

plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
plt.show()

basename = "plotCoboltRuntimeComparison"
plotFile = coboltOutputFolder / ('%s.pdf' % basename)
statisticsFile = coboltOutputFolder / ('%s.txt' % basename)

print("Write plots to %s" % getNormalizedPathName(plotFile))
figure.savefig(filename=str(plotFile))
plt.close()

print("Write statistics to %s" % getNormalizedPathName(statisticsFile))
writeBoxplotStatistics(bpl, statisticsFile, "Runtime [ms]")

# Cleanup
del bpl 
del basename
del plotFile
del statisticsFile

## Batch-dynamic link state modifications comparison

In [ ]:
coboltLinkStateModificationData = [aggregatedCoboltDataBatch[CB_LSM_ALL],
                                   aggregatedCoboltDataDynamic[CB_LSM_ALL],
                                   aggregatedCoboltDataDynamic[CB_LSM_TCA],
                                   aggregatedCoboltDataDynamic[CB_LSM_CEH],
                                  ]

ticks = ["Batch\n\n(N=%d)" % len(coboltLinkStateModificationData[0]),
         "Dynamic\n\n(N=%d)" % len(coboltLinkStateModificationData[1]),
         "Dynamic\nTC algorithm\n(N=%d)" % len(coboltLinkStateModificationData[2]),
         "Dynamic\ncontext events\n(N=%d)" % len(coboltLinkStateModificationData[3]),
         ]


w, h = plt.figaspect(.6)
figure = plt.figure(dpi=DPI_DEFAULT, figsize=(w, h))

bpl = plt.boxplot(coboltLinkStateModificationData, 
                  positions=np.array(range(len(coboltLinkStateModificationData))), 
                  sym='', widths=0.6, patch_artist=True)

setBoxplotColor(bpl, COLORS[0], subplotIndices=[0]) 
setBoxplotColor(bpl, COLORS[1], subplotIndices=[1,2,3]) 

# Draw temporary red and blue lines and use them to create a legend
# Temporary variable is necessary to avoid duplicate plots in Jupyter
_ = plt.plot([], c=COLORS[0], label='Batch')
_ = plt.plot([], c=COLORS[1], label='Dynamic')
plt.legend()

annotateBoxplot(bpl, figure.axes[0], labeledBoxplotElements=[BXP_MEDIANS], 
                relativePadding=3, valueFormatter="{:,.0f}")


axes = plt.axes()
axes.yaxis.grid()
plt.xticks(range(0, len(ticks)), ticks)
plt.xlim(-.5, len(ticks) - .5)
plt.ylabel('Num. link state modifications')

# New-style formatter: {[name]:[format]} where format is similar to the old-style format
plt.axes().yaxis.set_major_formatter(formatter=ticker.StrMethodFormatter('{x:,.0f}'))

plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
plt.show()

basename = "plotCoboltLinkStateModificationsBatchVsDynamic"
plotFile = coboltOutputFolder / ('%s.pdf' % basename)
statisticsFile = coboltOutputFolder / ('%s.txt' % basename)

print("Write plots to %s" % getNormalizedPathName(plotFile))
figure.savefig(filename=str(plotFile))
plt.close()

print("Write statistics to %s" % getNormalizedPathName(statisticsFile))
writeBoxplotStatistics(bpl, statisticsFile, "Execution time [ms]")

del bpl 
del basename
del plotFile
del statisticsFile

## Batch-dynamic execution time comparison

In [ ]:
runtimeDataBoxplot = [aggregatedCoboltDataBatch[CB_DURATION_TCC],
                      aggregatedCoboltDataDynamic[CB_DURATION_TCC],
                      aggregatedCoboltDataDynamic[CB_DURATION_TCA],
                      aggregatedCoboltDataDynamic[CB_DURATION_CEH],
                     ]

ticks = ["Batch\n\n(N=%d)" % len(runtimeDataBoxplot[0]),
         "Dynamic\n\n(N=%d)" % len(runtimeDataBoxplot[1]),
         "Dynamic:\nTC algorithm\n(N=%d)" % len(runtimeDataBoxplot[2]),
         "Dynamic:\ncontext events\n(N=%d)" % len(runtimeDataBoxplot[3]),
         ]

w, h = plt.figaspect(.6)
figure = plt.figure(dpi=DPI_DEFAULT, figsize=(w, h))

bpl = plt.boxplot(runtimeDataBoxplot, positions=np.array(range(len(runtimeDataBoxplot))), 
                  sym='', widths=0.4, patch_artist=True)

setBoxplotColor(bpl, COLORS[0], subplotIndices=[0]) 
setBoxplotColor(bpl, COLORS[1], subplotIndices=[1,2,3]) 

# Draw temporary red and blue lines and use them to create a legend
# Temporary variable is necessary to avoid duplicate plots in Jupyter
_ = plt.plot([], c=COLORS[0], label='Batch')
_ = plt.plot([], c=COLORS[1], label='Dynamic')
plt.legend(loc='upper left')

annotateBoxplot(bpl, figure.axes[0], labeledBoxplotElements=[BXP_MEDIANS], relativePadding=3, subplotIndices=[0,1,3])
annotateBoxplot(bpl, figure.axes[0], labeledBoxplotElements=[BXP_MEDIANS], relativePadding=11, subplotIndices=[2])


axes = plt.axes()
axes.yaxis.grid()
plt.xticks(range(0, len(ticks)), ticks)
plt.xlim(-.5, len(ticks) -.5)
plt.ylabel('Execution time [s]')

plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
plt.show()

basename = "plotCoboltRuntimeBatchVsDynamic"
plotFile = coboltOutputFolder / ('%s.pdf' % basename)
statisticsFile = coboltOutputFolder / ('%s.txt' % basename)

print("Write plots to %s" % getNormalizedPathName(plotFile))
figure.savefig(filename=str(plotFile))
plt.close()

print("Write statistics to %s" % getNormalizedPathName(statisticsFile))
writeBoxplotStatistics(bpl, statisticsFile, "Execution time [ms]")

del bpl 
del basename
del plotFile
del statisticsFile

## Batch-dynamic comparison per scenario

### Executin time and link state modification count vs. density

In [ ]:
def plotPerformanceXY(dimensionColumnName, dimensionAxisName, performanceMetricColumnName, performanceMetricAxisName,
                     yscale = None):
  xDataDynamicDensity = aggregatedCoboltDataDynamic[dimensionColumnName]
  yDataDynamicLSMs = aggregatedCoboltDataDynamic[performanceMetricColumnName]
  xDataBatchDensity = aggregatedCoboltDataBatch[dimensionColumnName]
  yDataBatchLSMs = aggregatedCoboltDataBatch[performanceMetricColumnName]

  figure = plt.figure(dpi=DPI_DEFAULT)
  lines = plt.plot(xDataBatchDensity, yDataBatchLSMs, 
                   xDataDynamicDensity, yDataDynamicLSMs,
                   markersize=5, linestyle='None')

  plt.setp(lines[0], color=COLORS[0], marker='x')
  plt.setp(lines[1], color=COLORS[1], marker='+')

  plt.xticks(range(0, int(max(xDataDynamicDensity)), 10))

  plt.legend(lines, ['Batch', 'Dynamic'], loc='upper left')

  plt.axes().yaxis.set_major_formatter(formatter=ticker.StrMethodFormatter('{x:,.0f}'))

  plt.xlabel(dimensionAxisName)
  plt.ylabel(performanceMetricAxisName)
  
  if yscale:
    plt.yscale(yscale)
  
  plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
  plt.show()
  
  basename = "plotXY%sVs%s" % (dimensionColumnName, performanceMetricColumnName)
  plotFile = coboltOutputFolder / ('%s.pdf' % basename)

  print("Write plots to %s" % getNormalizedPathName(plotFile))
  figure.savefig(filename=str(plotFile))
  plt.close()

plotPerformanceXY(CB_DENSITY_INITIAL, 'Initial density [mean mote degree]', CB_LSM_ALL, 'Num. link state modifications')
plotPerformanceXY(CB_DENSITY_INITIAL, 'Initial density [mean mote degree]', CB_DURATION_TCC, 'Execution time [ms]',
                 yscale='log')

### Execution time and link state modification count vs. scenario class

In [ ]:
def plotPerformanceByDimensions(dimensionColumnName, dimensionAxisName, performanceMetricColumnName, performanceMetricAxisName,
                               aspect=None):
  uniqueValues = sorted(aggregatedCoboltData[dimensionColumnName].unique())

  dataBatch = [(aggregatedCoboltDataBatch[aggregatedCoboltDataBatch[dimensionColumnName]==uniqueValue])[performanceMetricColumnName] for uniqueValue in uniqueValues]
  dataDynamic = [(aggregatedCoboltDataDynamic[aggregatedCoboltDataDynamic[dimensionColumnName]==uniqueValue])[performanceMetricColumnName] for uniqueValue in uniqueValues]

  if aspect:
    w, h = plt.figaspect(aspect)
    figure = plt.figure(dpi=DPI_DEFAULT, figsize=(w, h))
  else:
    figure = plt.figure(dpi=DPI_DEFAULT)

  boxplotPositions = np.array(range(len(uniqueValues)))
  bpl = plt.boxplot(dataBatch, positions=boxplotPositions*2.0-0.4, sym='', patch_artist=True)
  bpr = plt.boxplot(dataDynamic, positions=boxplotPositions*2.0+0.4, sym='', patch_artist=True)
  setBoxplotColor(bpl, COLORS[0]) 
  setBoxplotColor(bpr, COLORS[1])

  _ = plt.plot([], c=COLORS[0], label='Batch')
  _ = plt.plot([], c=COLORS[1], label='Dynamic')
  plt.legend(loc='upper left')

  annotateBoxplot(bpl, figure.axes[0])
  annotateBoxplot(bpr, figure.axes[0])

  mediansBatch = getBoxplotProperty(boxplot=bpl, property=BXP_MEDIANS)
  mediansDynamic = getBoxplotProperty(boxplot=bpr, property=BXP_MEDIANS)
  medianFractions = mediansBatch / mediansDynamic
  medianFractionsDict = {uniqueValues[i]: medianValues[0] for i, medianValues in enumerate(medianFractions)}

  ticks = ['%d\n(B/D: %.2f,N=%d)' % 
           (value, medianFractionsDict[value], len(dataBatch[i])) 
           for i, value in enumerate(uniqueValues)]
  plt.xticks(range(0, len(ticks) * 2, 2), ticks)
  plt.xlim(-1, len(ticks)*2-1)
  plt.ylim(0)
  plt.xlabel(dimensionAxisName)
  plt.ylabel(performanceMetricAxisName)
  plt.title("%s vs. %s" % (performanceMetricAxisName, dimensionAxisName) + '\n(B/D: batch vs. dynamic, N: data points per boxplot)')
  plt.axes().yaxis.set_major_formatter(formatter=ticker.StrMethodFormatter('{x:,.0f}'))

  plt.tight_layout(pad=DEFAULT_TIGHT_LAYOUT_PADDING)
  plt.show()
  
  
  basename = "plotPerDimension%sVs%s" % (dimensionColumnName, performanceMetricColumnName)
  plotFile = coboltOutputFolder / ('%s.pdf' % basename)
  statisticsFile = coboltOutputFolder / ('%s.txt' % basename)

  print("Write plots to %s" % getNormalizedPathName(plotFile))
  figure.savefig(filename=str(plotFile))
  plt.close()
  
  writeBoxplotStatistics(bpl, statisticsFile, performanceMetricAxisName)
  writeBoxplotStatistics(bpr, statisticsFile, performanceMetricAxisName, append=True)
  
plotData = [
  (CB_WORLD_SIZE, 'World size [m]', CB_LSM_ALL, 'Num. link state modifications'),
  (CB_MOTE_COUNT, 'Mote count', CB_LSM_ALL, 'Num. link state modifications'),
  (CB_DENSITY_INITIAL_BUCKET, 'Initial density [mean mote count]', CB_LSM_ALL, 'Num. link state modifications', .3),
  (CB_MOVEMENT_SPEED, 'Speed [m/s]', CB_LSM_ALL, 'Num. link state modifications'),
]

for plotProperties in plotData:
  plotPerformanceByDimensions(*plotProperties)

In [ ]:
plotData = [
  (CB_WORLD_SIZE, 'World size [m]', CB_DURATION_TCC, 'Execution time [ms]'),
  (CB_MOTE_COUNT, 'Mote count', CB_DURATION_TCC, 'Execution time [ms]'),
  (CB_DENSITY_INITIAL_BUCKET, 'Initial density [mean mote count]', CB_DURATION_TCC, 'Execution time [ms]', .3),
  (CB_MOVEMENT_SPEED, 'Speed [m/s]', CB_DURATION_TCC, 'Execution time [ms]'),
]

for plotProperties in plotData:
  plotPerformanceByDimensions(*plotProperties)

# eMoflon

## Prepare output folder

In [ ]:
emoflonOutputFolder = pathlib.Path('output') / 'emoflon'
emoflonOutputFolder.mkdir(exist_ok=True,parents=True)

## Coverage results (relevant projects)

In [ ]:
emoflonCoverageDataRelevantProjects = pd.read_csv('data/emoflon/coverageRelevantProjects.csv', sep=CSV_SEP)

emoflonCoverageDataRelevantProjects.sort_values(by='Coverage', inplace=True)

minCoverage = min(emoflonCoverageDataRelevantProjects['Coverage'])
maxCoverage = max(emoflonCoverageDataRelevantProjects['Coverage'])
sumCoveredInstructions = sum(emoflonCoverageDataRelevantProjects['Covered instructions'])
sumTotalInstructions = sum(emoflonCoverageDataRelevantProjects['Total instructions'])
totalCoverage = sumCoveredInstructions / sumTotalInstructions

registerLatexCommand(r"\numberEMoflonProjectsRelevant", num(len(emoflonCoverageDataRelevantProjects['Total instructions'])))
registerLatexCommand(r"\coverageEMoflonRelevant", siPercentage(totalCoverage * 100.0))
registerLatexCommand(r"\coverageEMoflonRelevantMin", siPercentage(minCoverage * 100.0))
registerLatexCommand(r"\coverageEMoflonRelevantMax", siPercentage(maxCoverage * 100.0))
registerLatexCommand(r"\coverageEMoflonRelevantInstructionsCovered", num(sumCoveredInstructions))
registerLatexCommand(r"\coverageEMoflonRelevantInstructionsAll", num(sumTotalInstructions))


summaryRow = pd.DataFrame([[createMidrule() + textbf("Summary"), totalCoverage, sumCoveredInstructions, sumTotalInstructions]], columns=emoflonCoverageDataRelevantProjects.columns)
emoflonCoverageDataRelevantProjects = emoflonCoverageDataRelevantProjects.append(summaryRow)

# Reformatting data frame
emoflonCoverageDataRelevantProjects['Coverage'] = \
  ["\\SI{%.1f}{\\percent}" % (100.0 * coverage) for coverage in emoflonCoverageDataRelevantProjects['Coverage']]
emoflonCoverageDataRelevantProjects['Covered instructions'] = \
  ["\\num{%d}" % (instructionCount) for instructionCount in emoflonCoverageDataRelevantProjects['Covered instructions']]
emoflonCoverageDataRelevantProjects['Total instructions'] = \
  ["\\num{%d}" % (instructionCount) for instructionCount in emoflonCoverageDataRelevantProjects['Total instructions']]
emoflonCoverageDataRelevantProjects['Project name'] = \
  [projectName.replace("org.moflon.sdm", "o.m.s") for projectName in emoflonCoverageDataRelevantProjects['Project name']]

# emoflonCoverageDataRelevantProjects.tail()

In [ ]:
colNames = ["Project name", "Coverage", r"\nInstructionsCovered", r"\nInstructionsTotal"]
formattedTable=tabulate(emoflonCoverageDataRelevantProjects, showindex=False,
               headers=[textbf(s) for s in colNames],
               stralign="left",
               tablefmt="latex_raw")

shortCaption = r"Coverage for \sdm-related \toolEMoflon projects"
caption = shortCaption + r" (Project name: o.m.s = org.moflon.sdm, \nInstructionsCovered/\nInstructionsTotal: number of covered/total instructions)"

formattedTable = convertToBooktabs(formattedTable)
formattedTable = compressWhitespace(formattedTable)
formattedTable = adjustAlignment(formattedTable, "lrrr")
formattedTable = encloseWithTableEnvironment(formattedTable, 
                                             caption=caption, 
                                             label="tabCoverageEMoflonRelevantProjects",
                                             shortCaption=shortCaption
                                            )

print(formattedTable)

emoflonCoverageDataRelevantProjectsOutput = emoflonOutputFolder / "coverageRelevantProjects.tex"

print("Write table to %s" % getNormalizedPathName(emoflonCoverageDataRelevantProjectsOutput))
with open(str(emoflonCoverageDataRelevantProjectsOutput), "w") as outputFile:
    outputFile.write(formattedTable)
    
del emoflonCoverageDataRelevantProjectsOutput
del formattedTable
del colNames
del caption
del shortCaption